In [1]:
from model_res2net50_v1b import res2net50_v1b as resnet50


In [2]:
def load_model(path):
    model = resnet50(num_classes=1)
    loaded = torch.load(path, map_location='cpu')
    state_dict=loaded.module.state_dict()
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Convert from pytorch to onnx

In [3]:
import torch

img_size = (200, 256)
batch_size = 1
onnx_model_path = 'model_nc_res2net.onnx'
# model = resnet50(num_classes=1, in_channels=1)
checkpoint_path = 'checkpoints/res2net50_v1b_trial1_epoch_6.pth'
# loaded = torch.load(checkpoint_path)
# state_dict=loaded.module.state_dict()
# model.load_state_dict(state_dict)
# model.eval()
model = load_model(checkpoint_path)
sample_input = torch.rand((batch_size, 1, *img_size))

y = model(sample_input)

torch.onnx.export(
    model,
    sample_input, 
    onnx_model_path,
    verbose=False,
    input_names=['input'],
    output_names=['output'],
    opset_version=12
)

# Convert from onnx to tensorflow

In [ ]:
import onnx
import onnx_tf
onnx_model_path = 'model_nc_res2net.onnx'
tf_model_path = 'model_nc_res2net_tf'
onnx_model = onnx.load(onnx_model_path)
# Convert ONNX model to TensorFlow format
tf_model = onnx_tf.backend.prepare(onnx_model)
# Export  TensorFlow  model 
tf_model.export_graph(tf_model_path)

In [5]:
#!pip install tensorflow==2.12 tensorflow-addons[tensorflow]


In [2]:
!pip list

Package                      Version
---------------------------- --------------
absl-py                      2.1.0
anyio                        4.1.0
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    1.0.3
attrs                        23.2.0
Babel                        2.14.0
backcall                     0.2.0
beautifulsoup4               4.12.3
bleach                       6.1.0
cachetools                   5.3.3
certifi                      2024.2.2
cffi                         1.16.0
charset-normalizer           3.3.2
cloudpickle                  3.0.0
comm                         0.2.2
debugpy                      1.8.1
decorator                    5.1.1
defusedxml                   0.7.1
dm-tree                      0.1.8
exceptiongroup               1.2.1
executing                    2.0.1
fastjsonschema               2.19.1

In [ ]:
#This is dependency list which works right now!

# Convert from tensorflow to tflite

In [ ]:
import tensorflow as tf

saved_model_dir = 'model_nc_res2net_tf'
tflite_model_path = 'model_nc_res2net.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [10]:
#Please pay a close attention to installed versions of packages; I had to do much experimentation to find the working combination. On the ai machine ai01, use 'export_to_tflite' environment

In [ ]:
saved_model_dir = 'model_nc_res2net_tf'  # Update with your actual path
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Enable TensorFlow ops in TFLite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]
converter.experimental_new_converter = True

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = 'converted_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
import tensorflow as tf
model_dir = 'model_nc_res2net_tf'
tflite_model_path = 'converted_model.tflite'

converter = tf.lite.TFLiteConverter.from_saved_model(model_dir)
converter.experimental_new_converter = True
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print('saved')